In [2]:
import pandas as pd 
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm 
import gc
import random
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# string 타입의 Time column 을 datetime 타입으로 변경해주는 함수
def make_datetime(x):
    x=str(x)
    year=int(x[:4])
    month=int(x[4:6])
    day=int(x[6:8])
    hour=int(x[8:10])
    min=int(x[10:12])
    sex=int(x[12:])
    return dt.datetime(year,month,day,hour)

In [4]:
# 정규방정식을 활용해 (,)( )과 같은 불필요한 데이터 정제
def string2num(x):
    x=re.sub(r"[^0-9]+",'',str(x))
    if x=='':
        return 0
    else:
        return int(x)

In [5]:
train_err=pd.read_csv('train_err_data.csv')
train_err['fwver'].unique()

array(['05.15.2138', '04.33.1185', '04.33.1261', '04.22.1750',
       '04.22.1778', '04.16.3553', '04.33.1149', '04.16.3571',
       '05.66.3237', '05.66.3571', '03.11.1149', '04.22.1684',
       '03.11.1167', '04.82.1684', '04.82.1778', '04.33.1171',
       '04.73.2237', '10', '04.82.1730', '04.73.2571', '8.5.3',
       '04.22.1666', '03.11.1141', '05.15.2120', '04.33.1125',
       '04.16.3439', '04.22.1442', '04.33.1095', '04.16.3569',
       '05.15.2090', '05.15.3104', '05.15.2122', '04.22.1656',
       '04.16.2641', '05.15.2114', '04.16.3345', '05.15.2092'],
      dtype=object)

In [6]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


### ID, model, fwvwer에 따라 각 errtype의 빈도를 측정하려한다.
### 확인 결과 test와 train의 errtype의 종류는 동일하다.
### 시간변수를 무시하고 ID, model, fwver에 의해서만 group을 만들어주기 위해 사용할 변수를 생성한다.

### 우선 연산의 간편성을 위해 model_nm과 fwver는 label encoding을 해준다(후에 decoding을 통해 원래 값으로 변환해준다).


In [7]:
from sklearn.preprocessing import LabelEncoder

# Labeling 'model_nm' & 'fwver'
encoder_model_tr=LabelEncoder()
encoder_model_tr.fit(train_err['model_nm'])
model_labeled_tr=encoder_model_tr.transform(train_err['model_nm'])
train_err['model_nm']=model_labeled_tr

encoder_fwver_tr=LabelEncoder()
encoder_fwver_tr.fit(train_err['fwver'])
fwver_labeled_tr=encoder_fwver_tr.transform(train_err['fwver'])
train_err['fwver']=fwver_labeled_tr

print('train_model_nm의 라벨 값:',train_err['model_nm'].unique())
print('train_fwver의 라벨값:',train_err['fwver'].unique())

train_model_nm의 라벨 값: [3 2 0 1 7 4 5 8 6]
train_fwver의 라벨값: [31 19 20 13 14  6 17  8 33 34  1 12  2 23 25 18 21 35 24 22 36 11  0 29
 16  5  9 15  7 26 32 30 10  3 28  4 27]


### group을 나누기 위한 새로운 변수 group을 생성한다.

- group=train_err['user_id']*1000+train_err['model_nm']*100+train_err['fwver'] 

-> 앞의 5자리는 id, 중간의 한 자리는 model을, 뒤의 2자리는 fwver을 나타낸다

In [8]:
train_err['group']=train_err['user_id']*1000 + train_err['model_nm']*100 + train_err['fwver']
unique_index_list = train_err['group'].unique()
print('새로 생성된 index의 종류:', end='')
print(unique_index_list)
    
print()

print('각 index의 갯수:')
print(train_err['group'].value_counts())

새로 생성된 index의 종류:[10000331 10001219 10001220 ... 24998013 24998014 24999331]

각 index의 갯수:
24934402    222186
20546331    195538
15570402    175730
20352402     83680
11041331     66093
             ...  
20331217         1
20099635         1
24435219         1
16010013         1
11157636         1
Name: group, Length: 24062, dtype: int64


### errtype별로 더하여 빈도수를 측정하기 위해서 errtype에 대해서 One-Hot 인코딩을 실시한다.

시간이 오래 걸리므로 코드는 주석처리를 했고, 따로 내보낸 csv파일을 사용한다.

In [9]:
train_err_incoded=pd.get_dummies(train_err,columns=['errtype'])
train_err_incoded.shape

(16554663, 47)

In [10]:
train_err_incoded.columns

Index(['user_id', 'time', 'model_nm', 'fwver', 'errcode', 'group', 'errtype_1',
       'errtype_2', 'errtype_3', 'errtype_4', 'errtype_5', 'errtype_6',
       'errtype_7', 'errtype_8', 'errtype_9', 'errtype_10', 'errtype_11',
       'errtype_12', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_16',
       'errtype_17', 'errtype_18', 'errtype_19', 'errtype_20', 'errtype_21',
       'errtype_22', 'errtype_23', 'errtype_24', 'errtype_25', 'errtype_26',
       'errtype_27', 'errtype_28', 'errtype_30', 'errtype_31', 'errtype_32',
       'errtype_33', 'errtype_34', 'errtype_35', 'errtype_36', 'errtype_37',
       'errtype_38', 'errtype_39', 'errtype_40', 'errtype_41', 'errtype_42'],
      dtype='object')

### group변수를 기준으로 groupby한 후 errtype별로 더해준다 

In [11]:
err_get=train_err_incoded.groupby(['group'])[ 'errtype_1',
       'errtype_2', 'errtype_3', 'errtype_4', 'errtype_5', 'errtype_6',
       'errtype_7', 'errtype_8', 'errtype_9', 'errtype_10', 'errtype_11',
       'errtype_12', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_16',
       'errtype_17', 'errtype_18', 'errtype_19', 'errtype_20', 'errtype_21',
       'errtype_22', 'errtype_23', 'errtype_24', 'errtype_25', 'errtype_26',
       'errtype_27', 'errtype_28', 'errtype_30', 'errtype_31', 'errtype_32',
       'errtype_33', 'errtype_34', 'errtype_35', 'errtype_36', 'errtype_37',
       'errtype_38', 'errtype_39', 'errtype_40', 'errtype_41', 'errtype_42'].sum()

# 주의: 용량이 커서 노트북으로는 실행되지 않는다.

MemoryError: Unable to allocate 5.06 GiB for an array with shape (41, 16554663) and data type int64

In [ ]:
err_get_train=pd.DataFrame(err_get)   # 다시 데이터 프레임으로 변환

In [ ]:
err_get_train.shape

In [ ]:
err_get_train.describe()

### err_get_train에 group별 시간min(), 시간max(), model_nm, fwver,user_id 삽입
같은 group에 속하는 데이터 내에서 시간의 최댓값(max value)과 최소값(min value)으로 구간을 구하고, 나중에 problem 파일의 time이 속하면 matching시켜 준다.

In [13]:
time_min_tr=train_err.groupby(['group'])['time'].min()
time_min_train=pd.DataFrame(time_min_tr)

time_max_tr=train_err.groupby(['group'])['time'].max()
time_max_train=pd.DataFrame(time_max_tr)

id_get_tr=train_err.groupby(['group'])['user_id'].min()
id_get_train=pd.DataFrame(id_get_tr)

model_get_tr=train_err.groupby(['group'])['model_nm'].min()
model_get_train=pd.DataFrame(model_get_tr)

fw_get_tr=train_err.groupby(['group'])['fwver'].min()
fw_get_train=pd.DataFrame(fw_get_tr)

In [14]:
print(time_min_train.shape, time_max_train.shape, id_get_train.shape,model_get_train.shape, fw_get_train.shape)

(24062, 1) (24062, 1) (24062, 1) (24062, 1) (24062, 1)


In [15]:
err_get_train['time_min']=time_min_train
err_get_train['time_max']=time_max_train
err_get_train['user_id']=id_get_train
err_get_train['model_nm']=model_get_train
err_get_train['fwver']=fw_get_train

In [1]:
err_get_train.head(5)

NameError: name 'err_get_train' is not defined

### model_nm과 fwver를 decoding해주므로 원래 형태로 변환한다.
일단 수치적 의미를 막기 위해 원래 형태로 전환해주는 것임

In [18]:
err_get_train['model_nm']=encoder_model_tr.inverse_transform(err_get_train['model_nm'])
err_get_train['fwver']=encoder_fwver_tr.inverse_transform(err_get_train['fwver'])

err_get_train

ValueError: y contains previously unseen labels: ['model_0' 'model_1' 'model_2' 'model_3' 'model_4' 'model_5' 'model_6'
 'model_7' 'model_8']

### index(group)을 제거해고 csv로 내보낸다.

In [19]:
err_get_train.to_csv(r"err_group_train.csv", index=False)

### 동일한 방식으로 test data도 형성해준다


In [3]:
test_err=pd.read_csv('test_err_data.csv')

In [4]:
#model, fwver labeling
from sklearn.preprocessing import LabelEncoder

encoder_model_te=LabelEncoder()
encoder_model_te.fit(test_err['model_nm'])
label_model_te=encoder_model_te.transform(test_err['model_nm'])
test_err['model_nm']=label_model_te


encoder_fwver_te=LabelEncoder()
encoder_fwver_te.fit(test_err['fwver'])
label_fwver_te=encoder_fwver_te.transform(test_err['fwver'])
test_err['fwver']=label_fwver_te

print('train_model_nm의 라벨 값:',test_err['model_nm'].unique())
print('train_fwver의 라벨값:',test_err['fwver'].unique())

train_model_nm의 라벨 값: [1 2 3 0 4 5 6 7 8]
train_fwver의 라벨값: [ 4  6 21 32 14 16 18  2 20 13 31 26 28 36 34 23 24 22 25 27  9 39 12 11
  1 19 35  3 29  0 10 33 17  8 30 37 38  5 15  7]


#### test에 fwver가 더 많이 존재함을 알 수 있다.

In [5]:
#group화 하기위한 group변수 생성
test_err['group']=test_err['user_id']*1000 + test_err['model_nm']*100 + test_err['fwver']
print(test_err['group'].unique())
print(test_err['group'].value_counts())

[30000104 30000106 30000221 ... 44997014 44997016 44998104]
41285332    396478
41591402    161516
41369402    159418
33363014     84166
30264221     83583
             ...  
44457636         1
32764220         1
36980218         1
39788013         1
34782013         1
Name: group, Length: 24096, dtype: int64


In [6]:
b=pd.get_dummies(test_err,columns=['errtype'])
b.shape

(16532648, 47)

In [7]:
b.columns

Index(['user_id', 'time', 'model_nm', 'fwver', 'errcode', 'group', 'errtype_1',
       'errtype_2', 'errtype_3', 'errtype_4', 'errtype_5', 'errtype_6',
       'errtype_7', 'errtype_8', 'errtype_9', 'errtype_10', 'errtype_11',
       'errtype_12', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_16',
       'errtype_17', 'errtype_18', 'errtype_19', 'errtype_20', 'errtype_21',
       'errtype_22', 'errtype_23', 'errtype_24', 'errtype_25', 'errtype_26',
       'errtype_27', 'errtype_28', 'errtype_30', 'errtype_31', 'errtype_32',
       'errtype_33', 'errtype_34', 'errtype_35', 'errtype_36', 'errtype_37',
       'errtype_38', 'errtype_39', 'errtype_40', 'errtype_41', 'errtype_42'],
      dtype='object')

In [8]:
err_get_te=b.groupby(['group'])[ 'errtype_1',
       'errtype_2', 'errtype_3', 'errtype_4', 'errtype_5', 'errtype_6',
       'errtype_7', 'errtype_8', 'errtype_9', 'errtype_10', 'errtype_11',
       'errtype_12', 'errtype_13', 'errtype_14', 'errtype_15', 'errtype_16',
       'errtype_17', 'errtype_18', 'errtype_19', 'errtype_20', 'errtype_21',
       'errtype_22', 'errtype_23', 'errtype_24', 'errtype_25', 'errtype_26',
       'errtype_27', 'errtype_28', 'errtype_30', 'errtype_31', 'errtype_32',
       'errtype_33', 'errtype_34', 'errtype_35', 'errtype_36', 'errtype_37',
       'errtype_38', 'errtype_39', 'errtype_40', 'errtype_41', 'errtype_42'].sum()

In [9]:
err_get_test=pd.DataFrame(err_get_te)
err_get_test.describe()

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
count,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,...,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000,24096.000000
mean,0.918036,0.866077,0.970369,49.236346,37.799676,2.341177,2.347361,0.008176,0.003818,6.183931,...,7.398821,6.521207,0.508549,0.391227,0.391974,0.197294,0.850639,30.211902,4.492198,1.425174
std,5.987339,34.696152,22.965019,2575.415079,181.956588,26.611904,26.496412,0.218978,0.085381,312.972711,...,8.364981,120.037581,8.240654,0.514206,0.515814,1.715862,11.330104,53.939188,14.392031,3.009669
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,12.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,11.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,38.000000,3.000000,1.000000
max,152.000000,2770.000000,1547.000000,396441.000000,14820.000000,1670.000000,1656.000000,17.000000,5.000000,41571.000000,...,350.000000,15777.000000,929.000000,15.000000,15.000000,106.000000,894.000000,1326.000000,1201.000000,18.000000


In [12]:
time_min_te=test_err.groupby(['group'])['time'].min()
time_min_test=pd.DataFrame(time_min_te)

time_max_te=test_err.groupby(['group'])['time'].max()
time_max_test=pd.DataFrame(time_max_te)


id_get_te=test_err.groupby(['group'])['user_id'].min()
id_get_test=pd.DataFrame(id_get_te)

model_get_te=test_err.groupby(['group'])['model_nm'].min()
model_get_test=pd.DataFrame(model_get_te)

fw_get_te=test_err.groupby(['group'])['fwver'].min()
fw_get_test=pd.DataFrame(fw_get_te)

In [13]:
print(time_min_test.shape,time_max_test.shape,id_get_test.shape,model_get_test.shape,fw_get_test.shape)

(24096, 1) (24096, 1) (24096, 1) (24096, 1) (24096, 1)


In [14]:
err_get_test['time_min']=time_min_test
err_get_test['time_max']=time_max_test
err_get_test['user_id']=id_get_test
err_get_test['model_nm']=model_get_test
err_get_test['fwver']=fw_get_test

In [15]:
err_get_test

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
group,,,,,,,,,,,,,,,,,,,,,
30000104,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,46.0,0.0,0.0,20201101030227,20201127024946,30000,1,4
30000106,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.0,0.0,1.0,20201118073855,20201122050431,30000,1,6
30000221,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,20201127094708,20201130192105,30000,2,21
30001332,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20201101042135,20201130221906,30001,3,32
30002014,0.0,0.0,0.0,0.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,14.0,95.0,1.0,0.0,20201101003616,20201126040300,30002,0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44996014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,156.0,1.0,0.0,20201101003242,20201119052538,44996,0,14
44996016,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,173.0,1.0,3.0,20201119053129,20201130220212,44996,0,16
44997014,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,...,3.0,0.0,28.0,44.0,0.0,20201101000032,20201127035735,44997,0,14


## model_nm과 fwver를 원래형태로 변환

In [16]:
err_get_test['model_nm']=encoder_model_te.inverse_transform(err_get_test['model_nm'])
err_get_test['fwver']=encoder_fwver_te.inverse_transform(err_get_test['fwver'])

err_get_test

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
group,,,,,,,,,,,,,,,,,,,,,
30000104,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,46.0,0.0,0.0,20201101030227,20201127024946,30000,model_1,04.16.3553
30000106,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.0,0.0,1.0,20201118073855,20201122050431,30000,model_1,04.16.3571
30000221,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,20201127094708,20201130192105,30000,model_2,04.33.1261
30001332,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20201101042135,20201130221906,30001,model_3,05.15.2138
30002014,0.0,0.0,0.0,0.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,14.0,95.0,1.0,0.0,20201101003616,20201126040300,30002,model_0,04.22.1750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44996014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,156.0,1.0,0.0,20201101003242,20201119052538,44996,model_0,04.22.1750
44996016,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,173.0,1.0,3.0,20201119053129,20201130220212,44996,model_0,04.22.1778
44997014,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,...,3.0,0.0,28.0,44.0,0.0,20201101000032,20201127035735,44997,model_0,04.22.1750


In [17]:
err_get_test.to_csv(r"err_group_test.csv", index=False)

### 위의 명령어를 일일이 다시 돌릴 필요 없게 저장한 excel에서 다시 데이터를 불러와 사용한다.

In [3]:
train=pd.read_csv(r"err_group_train.csv")

In [4]:
train

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24057,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750
24058,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778
24059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750
24060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778


In [5]:
test=pd.read_csv(r"err_group_test.csv")
test

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver
0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,46.0,0.0,0.0,20201101030227,20201127024946,30000,model_1,04.16.3553
1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,42.0,0.0,1.0,20201118073855,20201122050431,30000,model_1,04.16.3571
2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,1.0,20201127094708,20201130192105,30000,model_2,04.33.1261
3,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20201101042135,20201130221906,30001,model_3,05.15.2138
4,0.0,0.0,0.0,0.0,31.0,2.0,1.0,0.0,0.0,0.0,...,0.0,14.0,95.0,1.0,0.0,20201101003616,20201126040300,30002,model_0,04.22.1750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,156.0,1.0,0.0,20201101003242,20201119052538,44996,model_0,04.22.1750
24092,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,173.0,1.0,3.0,20201119053129,20201130220212,44996,model_0,04.22.1778
24093,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0,...,3.0,0.0,28.0,44.0,0.0,20201101000032,20201127035735,44997,model_0,04.22.1750
24094,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,5.0,0.0,20201127040356,20201130232034,44997,model_0,04.22.1778


In [7]:
y=test.groupby(['fwver'])['model_nm'].unique()
y

fwver
03.11.1141    [model_4]
03.11.1149    [model_4]
03.11.1167    [model_4]
04.16.3439    [model_1]
04.16.3553    [model_1]
04.16.3569    [model_1]
04.16.3571    [model_1]
04.22.1170    [model_0]
04.22.1448    [model_0]
04.22.1478    [model_0]
04.22.1608    [model_0]
04.22.1656    [model_0]
04.22.1666    [model_0]
04.22.1684    [model_0]
04.22.1750    [model_0]
04.22.1772    [model_0]
04.22.1778    [model_0]
04.33.1125    [model_2]
04.33.1149    [model_2]
04.33.1171    [model_2]
04.33.1185    [model_2]
04.33.1261    [model_2]
04.73.2237    [model_8]
04.73.2569    [model_8]
04.73.2571    [model_8]
04.73.2577    [model_8]
04.82.1684    [model_5]
04.82.1730    [model_5]
04.82.1778    [model_5]
05.15.2092    [model_3]
05.15.2114    [model_3]
05.15.2120    [model_3]
05.15.2138    [model_3]
05.15.3104    [model_3]
05.66.3237    [model_7]
05.66.3571    [model_7]
10            [model_6]
10.22.1770    [model_0]
10.22.1780    [model_0]
8.5.3         [model_6]
Name: model_nm, dtype: object

ID와 fwver에 의해서만 그룹화를 해도 똑같은 결과가 나온다. 


### target값을 matching시킨다. (id기준으로 시간을 merge)

In [8]:
train_pro=pd.read_csv(r"train_problem_data.csv")

In [9]:
TT=pd.merge(train,train_pro,on="user_id",how='outer')

In [10]:
train_t=pd.DataFrame(TT)

In [12]:
train_t

,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,...,errtype_39,errtype_40,errtype_41,errtype_42,time_min,time_max,user_id,model_nm,fwver,time
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,20201101025616,20201130212033,10000,model_3,05.15.2138,NaN
1,0.0,0.0,0.0,0.0,9.0,1.0,1.0,0.0,0.0,0.0,...,0.0,74.0,28.0,0.0,20201101020415,20201112030039,10001,model_2,04.33.1185,2.020111e+13
2,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,28.0,1.0,20201112030617,20201130222646,10001,model_2,04.33.1261,2.020111e+13
3,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,20201101030251,20201130211528,10002,model_3,05.15.2138,NaN
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,20201101182532,20201110072130,10003,model_2,04.33.1185,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24841,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,38.0,0.0,0.0,20201101071624,20201119031243,24997,model_0,04.22.1750,2.020112e+13
24842,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,8.0,5.0,20201119031831,20201130231434,24997,model_0,04.22.1778,2.020112e+13
24843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,20201101050758,20201103050737,24998,model_0,04.22.1750,2.020111e+13
24844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,20201103051329,20201130210420,24998,model_0,04.22.1778,2.020111e+13


In [14]:
train_t.to_csv(r"train_with_target.csv", index=False)

### Time열(target)을 0과 1로 변환해준다.

In [ ]:
# 우선 데이터를 불러오고
train_t  = pd.read_csv('train_with_target.csv.csv')

In [ ]:
# 결측치 0으로 대체
train_t['time']=train_t['time'].fillna(0)

In [ ]:
# time열(target)의 값이 동일한 id 내에서 min값과 max값 사이에 있으면 1로 처리.
train_t_=train_t.copy()   # 원본 데이터 보존
for x in range(24846):   # 24846보다 len()이 더 적절할듯 나중에 수정
    if train_t_['time_min'][x]<train_t_['time'][x]<train_t_['time_max'][x]:
        train_t_['time'][x]=1
    else:
        train_t_['time'][x]=0

In [ ]:
train_t_['time']

In [ ]:
# 동일한 user_id와 fwver에서 신고한 횟수를 구해준다.
train_t_sum=train_t_.groupby(['user_id', 'fwver'])['time'].sum()

In [ ]:
# time열의 값이 1이상이면 한 번이라도 신고를 했으면 해당 user_id와 fwver에서 신고를 했다는 것이므로 1로 처리.
train_t_sum_list=list(train_t_sum)
for x in range(24062):
    if T1[x]>0:
        T1[x]=1

In [ ]:
# 다시 데이터 프레임으로 전환
train_t_one_zero=pd.DataFrame(train_t_sum_list)

In [ ]:
# 동일한 user_id와 model_nm, fwver의 중복값을 제거
final_train_data=train_t_one_zero.drop_duplicates(['user_id','model_nm','fwver'])

In [ ]:
train_data.index=np.arange(24062)
train_data

In [ ]:
test_err= pd.read_csv('LG/err_group_test.csv')
test_err

In [ ]:
test_err2 = test_err
test_err2.shape

test data의 user_id 하나가 아예 없는데 수정 필요.

In [ ]:
# 해당 user_id의 정보를 만들기 위해 데이터를 생성해서 붙여준다.
additional_index = pd.DataFrame(np.zeros(len(test_err2.columns)).reshape(1, -1))
columns_names = test_err2.columns
additional_index.columns = columns_names
additional_index['model_nm'] = test_err2.model_nm.mode()
additional_index.user_id = [43262]   # 리스트로 입력 안 하면 오류 뜬다.
additional_index.index = [43262]
additional_index

In [ ]:
test_data = test_err2.append(additional_index)
test_data

In [ ]:
test_data_dup=final_test_data.drop_duplicates(['user_id','model_nm','fwver'])
test_data_dup

- model_nm도 수치적 의미가 있다고 얘기가 나오지 않아 우선 원-핫 인코딩 처리를 했다.

In [ ]:
final_train_data=pd.get_dummies(train_data,columns=['model_nm'])
final_train_data.shape

In [ ]:
final_test_data=pd.get_dummies(test_data.columns=['model_nm'])
final_test_data.shape

# Modeling

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #시각화 패키지
import seaborn as sns #시각화 패키지
plt.style.use('fivethirtyeight') # 테마바꾸기
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from tqdm import tnrange, tqdm_notebook, notebook
import time

from sklearn.preprocessing import StandardScaler, scale

from sklearn.linear_model import RidgeCV, Ridge, LassoCV, Lasso, LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [19]:
# 파일 불러오기(위의 명령어들을 수행하지 못해서 호재가 카톡방에 올린 데이터 사용.)
final_train_data = pd.read_csv('train_final.csv')
final_train_data.shape

(24062, 91)

In [20]:
final_test_data = pd.read_csv('test_final.csv')
final_test_data.shape

(24097, 90)

In [17]:
final_train_data.head(5)

,user_id,model_nm,fwver,target,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,...,quality_8,quality_9,quality_10,quality_11,quality_12,quality_0_minus,quality_1_minus,quality_2_minus,quality_6_minus,quality_11_minus
0,10000,model_3,05.15.2138,0,0,0,8,104,0,1,...,0.000000,0.000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,10001,model_2,04.33.1185,0,0,0,0,0,9,1,...,0.098146,0.000,967.026536,0.001984,0.023810,0.190007,0.190007,0.190007,0.190007,0.190007
2,10001,model_2,04.33.1261,1,0,0,0,0,44,0,...,0.185381,0.000,814.187848,0.001789,0.021471,0.216827,0.216827,0.216827,0.216827,0.216827
3,10002,model_3,05.15.2138,0,0,0,2,132,1,2,...,0.000000,0.125,3.875000,0.000000,0.000000,0.020833,0.020833,0.020833,0.020833,0.020833
4,10003,model_2,04.33.1185,0,0,0,0,0,1,1,...,0.098146,0.000,967.026536,0.001984,0.023810,0.190007,0.190007,0.190007,0.190007,0.190007


In [18]:
final_test_data.head(5)

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,...,quality_8,quality_9,quality_10,quality_11,quality_12,quality_0_minus,quality_1_minus,quality_2_minus,quality_6_minus,quality_11_minus
0,30000,model_1,04.16.3553,0,0,0,0,60,0,0,...,0.202465,2.049186,450.202025,0.000303,0.003631,0.178413,0.178413,0.178413,0.178413,0.178413
1,30000,model_1,04.16.3571,0,0,0,0,2,0,0,...,0.403315,0.000000,287.292818,0.000000,0.000000,0.162063,0.162063,0.162063,0.162063,0.162063
2,30000,model_2,04.33.1261,0,0,0,0,0,1,1,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,30001,model_3,05.15.2138,0,0,3,98,3,4,4,...,0.000000,0.000000,854.333333,0.000000,0.000000,0.250000,0.250000,0.250000,0.250000,0.250000
4,30002,model_0,04.22.1750,0,0,0,0,31,2,1,...,0.000000,0.000000,13.300000,0.000000,0.000000,0.125000,0.125000,0.125000,0.125000,0.125000


# Modeling

- train과 test 데이터에서 필요 없는 변수들 제거.

In [27]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = final_train_data.drop(["user_id","fwver", 'target'],axis=1)
train_y = final_train_data.target

In [28]:
test_x = final_test_data.drop(["user_id","fwver"],axis=1)

In [29]:
train_x.shape

(24062, 88)

In [30]:
test_x.shape

(24097, 88)

In [31]:
from sklearn.preprocessing import LabelEncoder

encoder_model_tr1.fit(train_x['model_nm'])
label_model_tr1=encoder_model_tr1.transform(train_x['model_nm'])
train_x['model_nm']=label_model_tr1

label_model_tr2=encoder_model_tr1.transform(test_x['model_nm'])
test_x['model_nm']=label_model_tr2

NameError: name 'encoder_model_tr1' is not defined

In [32]:
test_x.shape

(24097, 88)

In [33]:
#run traning
LGBM=LGBMClassifier(boosting_type='gbdt',
                    objective='binary', 
                    metric='auc', 
                    random_state=0, 
                    num_boost_round = 1000)
#result=cross_val_score(LGBM,train_features, train_target,cv=10,scoring='roc_auc')
#print('The cross validated score for LightBoost is:',result.mean())

In [ ]:
LGBM.fit(train_x, train_y)

In [ ]:
proba=pd.DataFrame(final_test_data['user_id'])
proba['problem']=LGBM.predict_proba(test_x)[:,1]
#LGBM=LGBM.drop([0,1],axis=1)
#LGBM.to_csv(r"/content/LGBM.csv", index=False)

In [ ]:
proba

In [ ]:
last2=proba.groupby(['user_id']).max()

In [ ]:
last

In [ ]:
last2.to_csv(r"LG_LGBM_5.csv", index=True)